### **Importing Libraries**

In [ ]:
import math
import random
from collections import defaultdict
from typing import List
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import deque
import copy
from Card_and_Deck import *
from GameState import GameState
from Game_and_Player import *
import time

### **MCTS Node Class**

In [ ]:
class MCTSNode:
    node_counter = 0
    def __init__(self, game_state, parent=None,move_taken=None,owner=None):
        self.game_state = game_state  # The game state at tho;his node
        self.parent = parent  # Parent node
        self.children = []  # List of child nodes
        self.visits = 1  # Number of times this node has been visited
        self.total_reward = 0  # Total reward (tricks won)
        self.untried_moves = list(self.game_state.get_legal_moves())  # Legal moves from this state
        self.move_taken = move_taken # move that led to this game state
        self.visited_nodes = {}
        self.owner=owner

    def expand(self,move):
          if move not in self.untried_moves:
            print(f"Warning: Move {move} not found in untried_moves!")
            print(f"Current untried_moves: {self.untried_moves}")
            return None


          game_state = self.game_state
          new_game_state = copy.deepcopy(game_state)
          self.untried_moves.remove(move)
          node_owner = self.game_state.players[self.game_state.current_player_index].name
          child_game_state = new_game_state.apply_move(move)

          child_node = MCTSNode(child_game_state, parent=self, move_taken=move,owner= node_owner)
          self.children.append(child_node)

          return child_node


    def simulate(self, game_state,current_name):

          game_state_new = copy.deepcopy(game_state)

          while not game_state_new.is_terminal():
              legal_moves = game_state_new.get_legal_moves()
              if not legal_moves:
                  break
              move = random.choice(legal_moves)
              game_state_new.apply_move(move)

          rewards = {}

          for player in game_state_new.players:
              player_tricks = game_state_new.tricks_won[player.name]
              rewards[player.name] = player_tricks/10
          return rewards


    def backpropagate(self, rewards):

          self.visits += 1

          if self.owner:
              reward = rewards.get(self.owner, 0)
              self.total_reward += reward

          if self.parent:
              self.parent.backpropagate(rewards)


    def ucb1(self,child, c=0.707):

        parent_visits = child.parent.visits

        if child.visits == 0:
            return float("inf")
        ucb_val= (child.total_reward / child.visits) + c * math.sqrt(2*math.log(parent_visits) / child.visits)
        return ucb_val


    def best_child(self, c=4):
        node = self
        if not self.children:
            print("No children found! Returning None.")
            return None
        best = max(node.children, key=lambda child: node.ucb1(child, c))
        return best



    def tree_policy(self, c=1.4):
          current_node = self
          depth = 0
          while not current_node.game_state.is_terminal():
              if current_node.untried_moves:
                  move = random.choice(current_node.untried_moves)
                  return current_node
              else:
                  current_node = current_node.best_child(c)  # v ← BESTCHILD(v, Cp)
                  depth += 1
          return current_node


### **MCTS Search Function**

In [ ]:

def mcts_search(root_node, num_simulations=100,c=0.707):

    game_state_new = root_node.game_state


    root = root_node
    current_name = root_node.game_state.players[root_node.game_state.current_player_index].name

    for _ in range(num_simulations):
        node = root
        root.visits += 1

        # Tree Traversal
        node = node.tree_policy(c)


        # Expansion
        if node.untried_moves:
            move = random.choice(node.untried_moves)
            node = node.expand(move)


        # Simulation
        reward = node.simulate(node.game_state,current_name)

        # Backpropagation
        node.backpropagate(reward)

    best_child = max(root_node.children, key=lambda child: child.total_reward / (child.visits + 1e-6))

    return best_child.move_taken


### **Determinizations**

In [ ]:


def uct_sampled_possible_worlds(game_state, trick_history, num_samples=20, max_steps=100,c=0.9):
    best_actions = []

    for _ in range(num_samples):
        new_game_state = game_state.redistribute_with_inference(trick_history)
        root_node = MCTSNode(new_game_state)
        best_action = mcts_search(root_node, max_steps,c)
        best_actions.append(best_action)

    best_move = max(best_actions, key=best_actions.count)
    return best_move


### **Random Vs UCT Imperfect**

In [ ]:
def evaluate_monte_carlo(num_games=1000, max_steps=100,num_samples=20,c=0.9,verbose=True):
    monte_carlo_wins = 0
    win_results = []

    for _ in range(num_games):

        player_names = ["Alice", "Bob", "Charlie"]
        roles = ["Teen", "Do", "Paanch"]
        index = [0, 1, 2]



        game = Game(player_names, roles, index)
        game_state = GameState(game)

        game_state.current_player_index=random.choice(index)
        monte_carlo_player = random.choice(index)
        if verbose:
            print(f"Monte Carlo Player Index: {monte_carlo_player}")
        cannot_have_suits = {name: set() for name in player_names}
        trick_history=[]
        trick=[]
        while not game_state.is_terminal():

            if game_state.current_player_index == monte_carlo_player:
                root_node = MCTSNode(game_state)
                best_move = uct_sampled_possible_worlds(game_state, trick_history, num_samples, max_steps,c)

            else:
                legal_moves1 = game_state.get_legal_moves()
                best_move = random.choice(legal_moves1)



            game_state.apply_move(best_move)
            trick.append([game_state.current_player_index,best_move])
            if game_state.table_cards==[]:
                trick_history.append(trick)
                trick=[]
        rewards = game_state.tricks_won
        if verbose:
           print(rewards)

        max_tricks = max(rewards.values())
        potential_winners = [name for name, count in rewards.items() if count == max_tricks]

        winner_name = potential_winners[0]
        monte_carlo_name = player_names[monte_carlo_player]



        if winner_name == monte_carlo_name:
            monte_carlo_wins += 1
            win_results.append(1)
            if verbose:
                print(f"Monte Carlo player won: {winner_name} ")
        else:
            win_results.append(0)
            if verbose:
               print(f"Monte Carlo player did not win: {winner_name} ")

    win_rate = monte_carlo_wins / num_games
    std_dev = np.std(win_results, ddof=1)

    print(f"Monte Carlo vs Random Win Rate: {win_rate:.2%}")
    print(f"Standard Deviation of Wins: {std_dev:.4f}")



In [ ]:
start=time.time()
evaluate_monte_carlo(num_games=100, num_samples=20,max_steps=100,c=0.9,verbose=True)
end=time.time()
print(f"Execution Time: {end - start:.4f} seconds")

Monte Carlo Player Index: 1
{'Alice': 1, 'Bob': 4, 'Charlie': 5}
Monte Carlo player did not win: Charlie 
Monte Carlo Player Index: 2
{'Alice': 6, 'Bob': 3, 'Charlie': 1}
Monte Carlo player did not win: Alice 
Monte Carlo Player Index: 1
{'Alice': 3, 'Bob': 2, 'Charlie': 5}
Monte Carlo player did not win: Charlie 
Monte Carlo Player Index: 2
{'Alice': 2, 'Bob': 3, 'Charlie': 5}
Monte Carlo player won: Charlie 
Monte Carlo Player Index: 0
{'Alice': 8, 'Bob': 0, 'Charlie': 2}
Monte Carlo player won: Alice 
Monte Carlo Player Index: 2
{'Alice': 1, 'Bob': 4, 'Charlie': 5}
Monte Carlo player won: Charlie 
Monte Carlo Player Index: 0
{'Alice': 5, 'Bob': 4, 'Charlie': 1}
Monte Carlo player won: Alice 
Monte Carlo Player Index: 1
{'Alice': 2, 'Bob': 7, 'Charlie': 1}
Monte Carlo player won: Bob 
Monte Carlo Player Index: 0
{'Alice': 3, 'Bob': 3, 'Charlie': 4}
Monte Carlo player did not win: Charlie 
Monte Carlo Player Index: 0
{'Alice': 6, 'Bob': 1, 'Charlie': 3}
Monte Carlo player won: Alice 


### **Rule Based Vs UCT Imperfect**

In [ ]:
def rule_based_move(game_state):
    legal_moves = game_state.get_legal_moves()
    return max(legal_moves, key=lambda card: card.rank)



def evaluate_monte_carlo_rule_based(num_games=1000, max_steps=100,num_samples=20,c=0.9,verbose=True):
    monte_carlo_wins = 0
    win_results = []

    for _ in range(num_games):

        player_names = ["Alice", "Bob", "Charlie"]
        roles = ["Teen", "Do", "Paanch"]
        index = [0, 1, 2]



        game = Game(player_names, roles, index)
        game_state = GameState(game)

        game_state.current_player_index=random.choice(index)
        monte_carlo_player = random.choice(index)
        if verbose:
            print(f"Monte Carlo Player Index: {monte_carlo_player}")
        cannot_have_suits = {name: set() for name in player_names}
        trick_history=[]
        trick=[]
        while not game_state.is_terminal():

            if game_state.current_player_index == monte_carlo_player:
                root_node = MCTSNode(game_state)
                best_move = uct_sampled_possible_worlds(game_state, trick_history, num_samples, max_steps,c)

            else:
                best_move = rule_based_move(game_state)



            game_state.apply_move(best_move)
            trick.append([game_state.current_player_index,best_move])
            if game_state.table_cards==[]:
                trick_history.append(trick)
                trick=[]
        rewards = game_state.tricks_won
        if verbose:
           print(rewards)

        max_tricks = max(rewards.values())
        potential_winners = [name for name, count in rewards.items() if count == max_tricks]

        winner_name = potential_winners[0]
        monte_carlo_name = player_names[monte_carlo_player]



        if winner_name == monte_carlo_name:
            monte_carlo_wins += 1
            win_results.append(1)
            if verbose:
                print(f"Monte Carlo player won: {winner_name} ")
        else:
            win_results.append(0)
            if verbose:
               print(f"Monte Carlo player did not win: {winner_name} ")

    win_rate = monte_carlo_wins / num_games
    std_dev = np.std(win_results, ddof=1)

    print(f"Monte Carlo vs Random Win Rate: {win_rate:.2%}")
    print(f"Standard Deviation of Wins: {std_dev:.4f}")



In [ ]:
start=time.time()
evaluate_monte_carlo_rule_based(num_games=100, max_steps=100,num_samples=20,c=0.9,verbose=True)
end=time.time()
print(f"Execution Time: {end - start:.4f} seconds")

Monte Carlo Player Index: 0
{'Alice': 3, 'Bob': 1, 'Charlie': 6}
Monte Carlo player did not win: Charlie 
Monte Carlo Player Index: 2
{'Alice': 1, 'Bob': 1, 'Charlie': 8}
Monte Carlo player won: Charlie 
Monte Carlo Player Index: 0
{'Alice': 4, 'Bob': 3, 'Charlie': 3}
Monte Carlo player won: Alice 
Monte Carlo Player Index: 2
{'Alice': 2, 'Bob': 5, 'Charlie': 3}
Monte Carlo player did not win: Bob 
Monte Carlo Player Index: 1
{'Alice': 2, 'Bob': 3, 'Charlie': 5}
Monte Carlo player did not win: Charlie 
Monte Carlo Player Index: 1
{'Alice': 6, 'Bob': 3, 'Charlie': 1}
Monte Carlo player did not win: Alice 
Monte Carlo Player Index: 0
{'Alice': 4, 'Bob': 3, 'Charlie': 3}
Monte Carlo player won: Alice 
Monte Carlo Player Index: 2
{'Alice': 3, 'Bob': 6, 'Charlie': 1}
Monte Carlo player did not win: Bob 
Monte Carlo Player Index: 1
{'Alice': 6, 'Bob': 0, 'Charlie': 4}
Monte Carlo player did not win: Alice 
Monte Carlo Player Index: 1
{'Alice': 3, 'Bob': 5, 'Charlie': 2}
Monte Carlo player wo